In [2]:
import pandas as pd

In [4]:
df = pd.read_excel('../Aramco Leased Vehicles - from 2012 up-to-date.xlsx')

In [18]:
df_filter = df[df['Pri. Stat.'] == 'Retired']
df_filter = df_filter.drop(columns=['IMEI Unit', 'Vehicle Door No', 'Data Collected on'])
df['service_year'] = df['service_year'].fillna(0).astype(int)
df_filter.head()

,Pri. Stat.,Make,Model,Model Year,Activity Type,Activity Type Description,Actual Retired Date,Fuel Type,service_year
0,Retired,NaN,NaN,2008,TLPB2,Carryall 4x4,03/31/2013,Gasoline,5
1,Retired,NaN,NaN,2008,TLPB2,Carryall 4x4,03/31/2013,Gasoline,5
2,Retired,NaN,NaN,2008,TLPB2,Carryall 4x4,03/31/2013,Gasoline,5
3,Retired,NaN,NaN,2008,TLPB2,Carryall 4x4,03/31/2013,Gasoline,5
4,Retired,NaN,NaN,2008,TLPB2,Carryall 4x4,03/31/2013,Gasoline,5


In [19]:
df_filter.dtypes

Pri. Stat.                   object
Make                         object
Model                        object
Model Year                    int64
Activity Type                object
Activity Type Description    object
Actual Retired Date          object
Fuel Type                    object
service_year                  int64
dtype: object

In [31]:
def get_survival_rate(df_filter):
    
    col1 = ['VehicleType', 'ModelYear', 'VehiclePurchase', 'FuelType']
    col2 = [f'{i}' for i in range(df_filter['service_year'].max()+1)]
    results = pd.DataFrame(columns=col1 + col2)

    for vehtype in df_filter['Activity Type Description'].unique():
        all_model_years = sorted(df_filter['Model Year'].unique())
        for modelyear in all_model_years:
            all_fuel_types= df_filter['Fuel Type'].unique()
            for fuel_type in all_fuel_types:
                data = df_filter[(df_filter['Activity Type Description'] == vehtype) & (df_filter['Model Year'] == modelyear) &(df_filter['Fuel Type'] == fuel_type)]
                if data.empty:
                    print(f'No data for vehicle type {vehtype}, model year {modelyear}, and tech type {fuel_type}')
                    new_row = {
                                'VehicleType': vehtype,
                                'ModelYear': modelyear,
                                'FuelType': fuel_type,
                                'VehiclePurchase': 0,
                            }
                    columns_after_vehicle_purchase = results.columns[results.columns.get_loc('VehiclePurchase') + 1:]
                    for col in columns_after_vehicle_purchase:
                        new_row[col] = "" 
                    results = pd.concat([results, pd.DataFrame([new_row])], ignore_index=True)
                else:
                    num_vehicle = data.shape[0]
                    min_service_year = data['service_year'].min()
                    max_service_year = data['service_year'].max()
                    survival_rate = [1 if i < min_service_year else 0 for i in range(data.shape[1])]
                    scrappage_num = 0
                    for i in range(data.shape[1]):
                        if i >= min_service_year and i < max_service_year:
                            scrappage_num += data.loc[data['service_year'] == i].shape[0]
                            survival_rate[i] = round((num_vehicle - scrappage_num) / num_vehicle, 3)
                    new_row = {
                        'VehicleType': vehtype,
                        'ModelYear': modelyear,
                        'FuelType': fuel_type,
                        'VehiclePurchase': num_vehicle,
                        **dict(zip(col2, survival_rate))
                    }
                    new_row_df = pd.DataFrame([new_row])  # Convert to DataFrame for concatenation
                    results = pd.concat([results, new_row_df], ignore_index=True)
    
    return results

In [32]:
results = get_survival_rate(df_filter)
results.head()

No data for vehicle type Carryall 4x4, model year 2005, and tech type Gasoline
No data for vehicle type Carryall 4x4, model year 2005, and tech type nan
No data for vehicle type Carryall 4x4, model year 2005, and tech type Diesel
No data for vehicle type Carryall 4x4, model year 2006, and tech type Gasoline
No data for vehicle type Carryall 4x4, model year 2006, and tech type nan
No data for vehicle type Carryall 4x4, model year 2006, and tech type Diesel
No data for vehicle type Carryall 4x4, model year 2007, and tech type Gasoline
No data for vehicle type Carryall 4x4, model year 2007, and tech type nan
No data for vehicle type Carryall 4x4, model year 2007, and tech type Diesel
No data for vehicle type Carryall 4x4, model year 2008, and tech type nan
No data for vehicle type Carryall 4x4, model year 2008, and tech type Diesel
No data for vehicle type Carryall 4x4, model year 2009, and tech type Gasoline
No data for vehicle type Carryall 4x4, model year 2009, and tech type nan
No dat

,VehicleType,ModelYear,VehiclePurchase,FuelType,0,1,2,3,4,5,...,7,8,9,10,11,12,13,14,15,16
0,Carryall 4x4,2005,0,,,,,,,,...,,,,,,,,,,
1,Carryall 4x4,2005,0,,,,,,,,...,,,,,,,,,,
2,Carryall 4x4,2005,0,,,,,,,,...,,,,,,,,,,
3,Carryall 4x4,2006,0,,,,,,,,...,,,,,,,,,,
4,Carryall 4x4,2006,0,,,,,,,,...,,,,,,,,,,


In [33]:
results.to_csv('../results/survival_rate.csv', index=False)